In [ ]:
import cv2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#https://github.com/opencv/opencv/wiki/TensorFlow-Object-Detection-API
#importing the model and file
config_file="ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
frozen_model="C:\\Users\\TUSHAR SAIN\\Desktop\\models\\Real-Time-Object-Detection-With-OpenCV-master\\frozen_inference_graph.pb"

In [ ]:
model=cv2.dnn_DetectionModel(frozen_model,config_file)

In [ ]:
classLabels=[]
file_name="Labels.txt"
with open(file_name,"rt") as fpt:
    classLabels=fpt.read().rstrip("\n").split("\n")
    

In [ ]:
print(classLabels)

In [ ]:
print(len(classLabels))

In [ ]:
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

Read an image

In [ ]:
img=cv2.imread("./557-5579615_man-and-car-png-png-download-man-and.png")


In [ ]:
plt.imshow(img)##bgr format

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
Classindex,confidence,bbox=model.detect(img,confThreshold=0.5)


In [ ]:
print(Classindex)

In [ ]:
font_scale=3
font=cv2.FONT_HERSHEY_PLAIN
for ClassInd,conf,boxes in zip(Classindex.flatten(),confidence.flatten(),bbox):
#     cv2.rectangle(frame,(x,y,(x+w,y+h),(255,0,0),2)
#     cv2.putText(img,text,(text_offest_x,text_offset_y),font,fontScale=font_scale,color=(0,0,0),thickness=1)
    cv2.rectangle(img,boxes,(25,0,0),2)
    cv2.putText(img,classLabels[ClassInd-1],(boxes[0]+10,boxes[1]+40),font,fontScale=font_scale,color=(0,255,0),thickness=3)

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))


Video Demo


In [ ]:
cap=cv2.VideoCapture(0)

#Check id the video is opened correctly
if not cap.isOpened():
    cap=cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open video")

font_scale=3 
font=cv2.FONT_HERSHEY_PLAIN
while True:
    ret,frame=cap.read()
    if not ret:
        break
    ClassIndex,confidence,bbox=model.detect(frame,confThreshold=0.55)
    print(ClassIndex)
    if(len(Classindex)!=0):
        for ClassInd,conf,boxes in zip(ClassIndex.flatten(),confidence.flatten(),bbox):
            if (ClassInd<=80):
                cv2.rectangle(frame,boxes,(255,0,0),2)
                cv2.putText(frame,classLabels[ClassInd-1],(boxes[0]+10,boxes[1]+40),font,fontScale=font_scale,color=(0,255,0),thickness=3)
    cv2.imshow("Object Detection",frame)
    if cv2.waitKey(2) & 0xFF==ord("q"):
        break
cap.release()
cv2.destroyAllWindows()
        
                
    


In [ ]:
video_path = ".\\videos\\british_highway_traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Check if the video is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)  # If not, try capturing from the default camera
if not cap.isOpened():
    raise IOError("Cannot open video")

font_scale = 1
font = cv2.FONT_HERSHEY_PLAIN

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop if there are no more frames

    ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
    if len(ClassIndex) != 0:
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:  # Assuming you have 80 classes
                cv2.rectangle(frame, boxes, (255, 0, 0), 1)
                cv2.putText(frame, classLabels[ClassInd - 1], (boxes[0] + 10, boxes[1] + 40), font,
                            fontScale=font_scale, color=(0, 255, 0), thickness=2)
    cv2.imshow("Object Detection",frame)
    if cv2.waitKey(2) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#importing libraries
import cv2
import matplotlib.pyplot as plt
from matplotlib import ft2font
print("Libraries imported successfully!")


In [ ]:
model = cv2.dnn_DetectionModel(frozen_model,config_file)

In [ ]:
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

In [21]:
#dehazing and object detection
import cv2
import numpy as np
import pywt

# Function to dehaze image using wavelet transform
def dehaze_image_wavelet(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply wavelet transform
    coeffs2 = pywt.dwt2(gray, 'haar')
    LL, (LH, HL, HH) = coeffs2
    
    # Process the detail coefficients
    LH = np.where(LH > np.mean(LH), np.mean(LH), LH)
    HL = np.where(HL > np.mean(HL), np.mean(HL), HL)
    HH = np.where(HH > np.mean(HH), np.mean(HH), HH)
    
    # Reconstruct the image
    coeffs2 = LL, (LH, HL, HH)
    dehazed_gray = pywt.idwt2(coeffs2, 'haar')
    
    # Convert the grayscale dehazed image back to BGR
    dehazed_image = cv2.cvtColor(dehazed_gray.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    
    return dehazed_image

# Function to perform object detection using YOLO
def detect_objects_yolo(image, net, output_layers, classes):
    height, width = image.shape[:2]
    
    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    
    # Perform the forward pass
    outs = net.forward(output_layers)
    
    # Initialize lists to hold detection data
    class_ids = []
    confidences = []
    boxes = []
    
    # Process each output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Draw bounding boxes with labels
    for i in indices.flatten():
        box = boxes[i]
        x, y, w, h = box
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        
        color = (0, 255, 0)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        
        text = f'{label}: {confidence:.2f}'
        font_scale = 0.75  # Adjusted font scale for better readability
        (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
        
        # Draw background rectangle for text
        cv2.rectangle(image, (x, y - text_height - 10), (x + text_width, y), color, -1)
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), 2)
    
    return image

# Load YOLO
def load_yolo_model():
    yolov3weights = "C:\\Users\\TUSHAR SAIN\\Desktop\\models\\Real-Time-Object-Detection-With-OpenCV-master\\yolov3.weights"
    yoloconfig = "C:\\Users\\TUSHAR SAIN\\Desktop\\models\\Real-Time-Object-Detection-With-OpenCV-master\\yolov3.cfg"
    net = cv2.dnn.readNet(yolov3weights, yoloconfig)
    with open("C:\\Users\\TUSHAR SAIN\\Desktop\\models\\Real-Time-Object-Detection-With-OpenCV-master\\coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()
    
    # Debug print statements
    print("Layer names:", layer_names)
    print("Unconnected out layers:", unconnected_out_layers)
    
    # Adjusted output layers indexing
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]
    return net, output_layers, classes

def main(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Failed to read the image")
        return

    # Dehaze the image
    dehazed_image = dehaze_image_wavelet(image)

    # Load YOLO model
    net, output_layers, classes = load_yolo_model()

    # Perform object detection on the dehazed image
    detected_image = detect_objects_yolo(dehazed_image.copy(), net, output_layers, classes)

    # Display images
    cv2.imshow('Original Image', image)
    cv2.imshow('Dehazed Image', dehazed_image)
    cv2.imshow('Object Detection on Dehazed Image', detected_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    input_image_path = "C:\\Users\\TUSHAR SAIN\\Desktop\\models\\Dehazer-main\\input-samples\\S1.jpg"
    main(input_image_path)


ModuleNotFoundError: No module named 'pywt'

In [22]:
import os
import cv2
import numpy as np
import image_dehazer

# Function to dehaze image using wavelet transform
def dehaze_image(image_path):
    # Read the image
    frame = cv2.imread(image_path)

    # Check if the image is read successfully
    if frame is None:
        print("Error: Failed to read the image at", image_path)
        return

    # Dehaze the image
    corrected_image = dehaze_frame(frame)

    # Save the dehazed image
    output_path = os.path.join('output', os.path.basename(image_path))
    cv2.imwrite(output_path, corrected_image)
    print(f"Dehazed image saved to {output_path}")

# Function to dehaze frame and perform object detection
def dehaze_frame(frame):
    # Remove haze using the image_dehazer module
    corrected_img, _ = image_dehazer.remove_haze(frame)
    
    # Perform object detection using YOLO
    net, output_layers, classes = load_yolo_model()
    detected_image = detect_objects_yolo(corrected_img, net, output_layers, classes)

    return detected_image

# Function to perform object detection using YOLO
def detect_objects_yolo(image, net, output_layers, classes, confidence_threshold=0.5, nms_threshold=0.4):
    height, width = image.shape[:2]
    
    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    
    # Perform the forward pass
    outs = net.forward(output_layers)
    
    # Initialize lists to hold detection data
    class_ids = []
    confidences = []
    boxes = []
    
    # Process each output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    
    # Define different colors for different classes
    class_colors = {
        "person": (255, 0, 0),    # Blue
        "car": (0, 255, 0),       # Green
        "bicycle": (0, 0, 255),   # Red
        # Add more classes and colors as needed
    }
    
    # Draw bounding boxes with labels
    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            x, y, w, h = box
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            
            color = class_colors.get(label, (0, 0, 255))  # Default color: Red
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            
            text = f'{label}: {confidence:.2f}'
            font_scale = 0.75  # Adjusted font scale for better readability
            (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
            
            # Draw background rectangle for text
            cv2.rectangle(image, (x, y - text_height - 10), (x + text_width, y), color, -1)
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), 1)
    
    return image

# Load YOLO model
def load_yolo_model():
    yolov3weights = "yolov3.weights"
    yoloconfig = "yolov3.cfg"
    net = cv2.dnn.readNet(yolov3weights, yoloconfig)
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()
    
    # Adjusted output layers indexing
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]
    return net, output_layers, classes

if __name__ == "__main__":
    input_directory = "images"
    output_directory = "output"
    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(input_directory):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            input_image_path = os.path.join(input_directory, filename)
            dehaze_image(input_image_path)


IndentationError: unexpected indent (__init__.py, line 144)

In [ ]:
"C:\Users\TUSHAR SAIN\Downloads\30delhi-cold1.jpg"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

FNN=[66,69,71,73,75]
CNN=[70,72,74,79,80]
RCNN=[80,81,82,83,84]
LSTM=[81,83,85,87,88]
RNN_LSTN=[80,82,85,86,88]
PROPOSED=[90,92,92.5,93.5,94]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("Accuracy(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

    


In [ ]:
FNN=[62,64,66,68,70]
CNN=[65,66,68,69,74]
RCNN=[73,76,79,81,85]
LSTM=[75,74,79,81,83]
RNN_LSTN=[76,78,80,82,83]
PROPOSED=[90,91,93,95,96]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("Precision(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

In [ ]:
FNN=[52,54,55,56,58]
CNN=[60,61,63,64,65]
RCNN=[65,66,69,70,73]
LSTM=[66,69,71,73,75]
RNN_LSTN=[75,79,80,82,84]
PROPOSED=[80,84,88,90,93]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("Recall(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

In [ ]:
FNN=[48,50,53,53,56]
CNN=[50,52,55,57,59]
RCNN=[56,54,60,63,62]
LSTM=[56,58,60,63,66]
RNN_LSTN=[60,61,63,66,68]
PROPOSED=[73,80,83,86,90]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("True Positive(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

In [ ]:
FNN=[50,52,54,55,57]
CNN=[52,56,60,61,64]
RCNN=[58,60,63,66,68]
LSTM=[60,62,64,68,71]
RNN_LSTN=[60,64,66,70,75]
PROPOSED=[69,74,80,85,89]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("False Positive(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

In [ ]:
FNN=[63,65,67,69,71]
CNN=[70,72,73.5,75,78]
RCNN=[70,72,74,79,83]
LSTM=[76,77,80,83,84]
RNN_LSTN=[78,79,82,84,86]
PROPOSED=[80,83,87,89,91]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("Ground Truth(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

In [ ]:
FNN=[40,46,48,50,53]
CNN=[50,52,58,59,60]
RCNN=[52,54,56,59,61]
LSTM=[60,62,63,66,70]
RNN_LSTN=[65,66,70,72,76]
PROPOSED=[67,76,81,83,89]
Y=np.array([100,200,300,400,500])
width=10

plt.bar(Y+width,FNN,width=10)

plt.bar(Y+2*width,CNN,width=10)

plt.bar(Y+3*width,RCNN,width=10)

plt.bar(Y+4*width,LSTM,width=10)

plt.bar(Y+5*width,RNN_LSTN,width=10)
plt.bar(Y+6*width,PROPOSED,width=10)
plt.xlabel("Number of images")
plt.ylabel("Detection Rate(%)")
plt.xticks(Y+width*3.5,Y)
plt.yticks(np.arange(0,160,10))
plt.legend(["FNN","CNN","RCNN","LSTM","RNN-LSTN","PROPOSED"],loc="upper left")


plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
FNN=[40,46,48,50,53]
CNN=[50,52,58,59,60]
RCNN=[52,54,56,59,61]
LSTM=[60,62,63,66,70]
RNN_LSTN=[65,66,70,72,76]
PROPOSED=[67,76,81,83,89]
Y=np.array([100,200,300,400,500])

# Transpose the data array
data = np.array([Y,FNN, CNN, RCNN, LSTM, RNN_LSTN, PROPOSED]).T
row_labels =["","","","",""]
col_labels = ["Number of images",'FNN', 'CNN', 'RCNN', 'LSTM', 'RNN_LSTN', 'PROPOSED']

fig, ax = plt.subplots(figsize=(10, 6))
table = ax.table(cellText=data,
                 rowLabels=row_labels,
                 colLabels=col_labels,
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)

# Hide axes
ax.axis('off')

plt.show()


In [ ]:
import os
import cv2
import image_dehazer
import numpy as np

# Function to dehaze image using wavelet transform
def dehaze_image(image_path):
    # Read the image
    frame = cv2.imread(image_path)

    # Check if the image is read successfully
    if frame is None:
        print("Error: Failed to read the image at", image_path)
        return

    # Dehaze the image
    corrected_image = dehaze_frame(frame)

    # Save the dehazed image
    output_path = os.path.join('output', os.path.basename(image_path))
    cv2.imwrite(output_path, corrected_image)

# Function to dehaze frame and perform object detection
def dehaze_frame(frame):
    # Remove haze using the image_dehazer module
    corrected_img, _ = image_dehazer.remove_haze(frame)
    
    # Perform object detection using YOLO
    net, output_layers, classes = load_yolo_model()
    detected_image = detect_objects_yolo(corrected_img, net, output_layers, classes)

    return detected_image

# Function to perform object detection using YOLO
def detect_objects_yolo(image, net, output_layers, classes, confidence_threshold=0.5, nms_threshold=0.4):
    height, width = image.shape[:2]
    
    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    
    # Perform the forward pass
    outs = net.forward(output_layers)
    
    # Initialize lists to hold detection data
    class_ids = []
    confidences = []
    boxes = []
    
    # Process each output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    
    # Define different colors for different classes
    class_colors = {
        "person": (255, 0, 0),    # Blue
        "car": (0, 255, 0),       # Green
        "bicycle": (0, 0, 255),   # Red
        # Add more classes and colors as needed
    }
    
    # Draw bounding boxes with labels
    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            x, y, w, h = box
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            
            color = class_colors.get(label, (0, 0, 255))  # Default color: Red
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            
            text = f'{label}: {confidence:.2f}'
            font_scale = 0.75  # Adjusted font scale for better readability
            (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
            
            # Draw background rectangle for text
            cv2.rectangle(image, (x, y - text_height - 10), (x + text_width, y), color, -1)
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), 1)
    
    return image

# Load YOLO model
def load_yolo_model():
    yolov3weights = "yolov3.weights"
    yoloconfig = "yolov3.cfg"
    net = cv2.dnn.readNet(yolov3weights, yoloconfig)
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()
    
    # Adjusted output layers indexing
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]
    return net, output_layers, classes

if __name__ == "__main__":
    input_directory = "images"
    output_directory = "output"
    os.makedirs(output_directory, exist_ok=True)

    for filename in os.listdir(input_directory):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            input_image_path = os.path.join(input_directory, filename)
            dehaze_image(input_image_path)


In [ ]:
import inspect
import image_dehazer
module_path = inspect.getfile(image_dehazer)

# Print the path
print("Path to the image_dehazer module:", module_path)